id: household

Choices:  
- heinz41  
- heinz32  
- heinz28  
- hunts32  

variables  
- disp (1/0)  
- feat (1/0)  
- price (Log?)  

Loading the dataset

In [2]:
import pickle
dataset = pickle.load(open("datasets/n 1000 - seed 1234 - time 2019-06-19-154935.pickle","rb"))

Describing the data

In [3]:
print("Number of datasets generated is %d \n" % dataset['n'])

print("Coefficients")
print(dataset['c'])

print("\n Data preview")
dataset['data'][0].head()

Number of datasets generated is 1000 

Coefficients
[0.8, 0.2, -0.5, 0.9, 1.1, -0.5, 0.1, 0.5, 0.25]

 Data preview


,id,disp.heinz41,disp.heinz32,disp.heinz28,disp.hunts32,feat.heinz41,feat.heinz32,feat.heinz28,feat.hunts32,price.heinz41,price.heinz32,price.heinz28,price.hunts32,choice,choiceindex
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz32,1
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,heinz32,1
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,heinz32,1
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,hunts32,3
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,heinz41,0


Running the model

In [2]:
from src.main import Mixedlogit
import pickle

dataset = pickle.load(open("datasets/n 1000 - seed 1234 - time 2019-06-19-154935.pickle","rb"))

mixedlogit = Mixedlogit(dataset)

mixedlogit.fit(drawtype="pseudo", n_draws=1, subset=0.001, verbose=1, method="Nelder-Mead")

Fitting 1 datasets 

Fitting model 1 



,a h41,a h32,a h28,m disp,m feat,m price,s disp,s feat,s price,MAPE,RMSE
0,0.490,0.711,-0.085,0.558,0.961,-0.194,2.051,2.153,0.615,324.072,0.909


,a h41,a h32,a h28,m disp,m feat,m price,s disp,s feat,s price,MAPE,RMSE
1,1.048,-0.111,-0.549,0.781,0.997,-0.578,0.640,0.706,0.602,106.313,0.269


,a h41,a h32,a h28,m disp,m feat,m price,s disp,s feat,s price,MAPE,RMSE
2,0.959,-0.075,-0.528,0.840,1.017,-0.730,0.576,0.638,0.646,98.339,0.251


,a h41,a h32,a h28,m disp,m feat,m price,s disp,s feat,s price,MAPE,RMSE
3,0.822,0.041,-0.327,1.000,1.082,-0.592,0.468,0.526,0.632,74.936,0.199


...

,a h41,a h32,a h28,m disp,m feat,m price,s disp,s feat,s price,MAPE,RMSE
37,0.807,0.108,-0.511,1.031,1.103,-0.546,0.000,0.085,0.000,39.531,0.174


,a h41,a h32,a h28,m disp,m feat,m price,s disp,s feat,s price,MAPE,RMSE
38,0.807,0.108,-0.511,1.031,1.103,-0.546,0.000,0.085,0.000,39.531,0.174


,a h41,a h32,a h28,m disp,m feat,m price,s disp,s feat,s price,MAPE,RMSE
39,0.807,0.108,-0.511,1.031,1.103,-0.546,0.000,0.085,0.000,39.531,0.174


Optimization done, time elapsed: 0:00:13


   duration: 13.367718935012817
        fun: 3387.289457521607
   hess_inv: array([[ 4.30656805e-03,  1.28693050e-04,  2.11474500e-03,
        -1.46208130e-03, -5.39720425e-04, -2.07442121e-03,
        -2.93575806e-01,  3.15437423e-02, -2.53790689e-01],
       [ 1.28693050e-04,  3.91600858e-03,  2.82462935e-03,
         2.84805926e-05, -2.27420522e-03,  1.18471157e-03,
        -5.21039418e-01,  4.65850114e-02, -4.49740458e-01],
       [ 2.11474500e-03,  2.82462935e-03,  8.73019036e-03,
        -1.50555293e-03, -4.16609651e-03, -4.61053444e-04,
        -7.73715362e-01,  7.14355375e-02, -6.68091450e-01],
       [-1.46208130e-03,  2.84805926e-05, -1.50555293e-03,
         7.94167968e-03, -1.33172593e-03,  1.36979536e-03,
        -9.21600967e-02,  7.12005812e-03, -8.00628372e-02],
       [-5.39720425e-04, -2.27420522e-03, -4.16609651e-03,
        -1.33172593e-03,  1.13155602e-02, -4.64659545e-04,
         3.76864245e-01, -3.63116493e-02,  3.24423859e-01],
       [-2.07442121e-03,  1.184711